<img align=center src="https://csc.web.cern.ch/wp-content/uploads/CSC.png" width=400/>

# *Solution*: The Map-Reduce Pattern of Parallelism
<hr style="border-top-width: 4px; border-top-color: #34609b;">

This exercise aims to illustrate the characteristics of the Map-Reduce parallel pattern by means of the Spark programming model.

## Prelude: Introduction to Spark

<img align=left src="http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png" width=150/>


[Spark](http://spark.apache.org) is a framework for parallel distributed data processing on a set of commodity machines. It frees the user from duties like task scheduling, data transfer and fault tolerance, so that they can focus on programming.

The Spark programming model uses the Map-Reduce model as a basis, simplifying its usage and extending it with more functionality. The following are two basic concepts in Spark:
* `Resilient Distributed Datasets (RDDs)`: an RDD is a distributed collection of items. The collection is divided in partitions, and these partitions are the units of parallelism.
* `Transformations and actions`: the user can apply either transformations or actions on RDDs. Transformations are `lazy`, they are not applied right away, unlike actions. In other words, one can apply a chain of transformations to an RDD, but only when appending an action to that chain the whole computation graph will start executing. `map` is an example of a transformation, while `reduce` is an action. However, Spark is not limited to just map and reduce: it offers a rich variety of functional-style operators. Although we will not use all those operators in this exercise, you can find a complete list [here](http://spark.apache.org/docs/latest/programming-guide.html).

As for the language, Spark programs can be developed in Scala, Java and Python. The latter will be used in this exercise.

## 1. Map-Reduce on a collection of elements

Map-Reduce is a combination of two patterns of parallelism:
* `Map`: given a collection of elements of type $X$, apply a function $F_{map}$ to each element that returns a single element of type $Y$.
* `Reduce`: given a collection of elements of type $Y$, combine its elements into an element of type $Y$ using a function $F_{reduce}$.

These two patterns can be chained to obtain a single result from a collection of elements:

$(X_{1},~X_{2},~...,~X_{n})~\xrightarrow{F_{map}}~(Y_{1},~Y_{2},~...,~Y_{n})~\xrightarrow{F_{reduce}}~Y_{result}$

Spark implements these patterns via two operators, [map](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.map.html) and [reduce](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.reduce.html). Both receive a function as parameter; in the case of `map`, the function must receive **one** parameter of type $X$ and return a value of type $Y$; regarding `reduce`, the function must receive **two** parameters of type $Y$ and return a value of type $Y$.

### First example

Let's start with a simple example of how to run a Map-Reduce chain with Spark.

First we will perform some initialisation of the Spark library. In particular, we will create a `SparkContext` object that we will use later.

In [1]:
from pyspark import SparkContext
sc = SparkContext()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/cvmfs/sft.cern.ch/lcg/releases/spark/3.2.1-cern1-84a1d/x86_64-centos7-gcc8-opt/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/cvmfs/sft.cern.ch/lcg/releases/hadoop/3.3.0-f6230/x86_64-centos7-gcc8-opt/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-08-29 13:57:24,496 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Next we will create our first RDD, i.e. a collection of elements in Spark. In this case, the collection is a list of numbers.

In [2]:
rdd = sc.parallelize([1, 2, 4, 8])

Let's now `map` a function to our collection to increment each of its elements. We will first define such function:

In [3]:
def increment(x):
    return x + 1

We can now apply the `increment` function to every element in our collection via the `map` operator.

<img align=left src="https://cdn.rawgit.com/jkthompson/pyspark-pictures/master/images/pyspark-page3.svg" width=500 height=250 />

In [4]:
incrementedRDD = rdd.map(increment)

You can actually check the incremented values with the [collect](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.collect.html?highlight=collect#pyspark.RDD.collect) function of Spark. Note that with the statement above you just *scheduled* a calculation which is not executed by the Spark runtime: `map` is indeed a *transformation*. The actual work is triggered by an *action*, such as `collect` or `reduce`.

In [5]:
incrementedRDD.collect()

[2, 3, 5, 9]

Let's suppose we would like to obtain the sum of all the numbers in the collection. For that purpose, we can use `reduce` and a function that returns the sum of two numbers.

In [6]:
def add(x, y):
    return x + y

Spark's `reduce` functionality will apply `add` to our collection, adding its elements two by two until calculating the total sum.

<img align=left src="https://cdn.rawgit.com/jkthompson/pyspark-pictures/master/images/pyspark-page23.svg" width=500 height=250 />

In [7]:
totalSum = incrementedRDD.reduce(add)
totalSum

19

Note that you can also do the same calculation in a single Map-Reduce chain:

In [8]:
rdd.map(increment).reduce(add)

19

### Exercise to complete

Now that you became familiar with the Map-Reduce pattern and its implementation in Spark, we propose you an exercise to apply what you just learned to a different problem.

In this case, our collection will be a list of town names:

In [9]:
names = sc.parallelize(['A Coruña', 'Aachen', 'Aarhus', 'Abbeville', 'Aberdeen', 'Adelboden', 'Agde', 'Agen', 'Agios Nikolaos', 'Agrigento', 'Agropoli', 'Aigues-Mortes', 'Aix-en-Provence', 'Aix-les-Bains', 'Ajaccio', 'Alanya', 'Alaró', 'Albacete', 'Albenga', 'Albi', 'Albufeira', 'Alcudia', 'Alessandria', 'Ålesund', 'Algeciras', 'Alghero', 'Alicante', 'Alkmaar', 'Alpbach', 'Alpe d\'Huez', 'Alta Badia', 'Altea', 'Amalfi', 'Amersfoort', 'Amiens', 'Amsterdam', 'Ancona', 'Andalo', 'Andermatt', 'Andorra la Vella', 'Andratx', 'Andria', 'Angers', 'Angoulême', 'Ankara', 'Annecy', 'Antalya', 'Antibes', 'Antwerp', 'Anzio', 'Aosta', 'Apeldoorn', 'Arcachon', 'Arenzano', 'Arezzo', 'Argostoli', 'Arlberg', 'Arles', 'Armagh', 'Arnhem', 'Arosa', 'Arras', 'Arrecife', 'Artà', 'Ascoli Piceno', 'Ascona', 'Asti', 'Athens', 'Athlone', 'Augsburg', 'Auxerre', 'Aveiro', 'Avellino', 'Avignon', 'Avoriaz', 'Axamer Lizum', 'Ayia Napa', 'Bad Gastein', 'Bad Hofgastein', 'Baden', 'Baiona', 'Bandol', 'Bar', 'Barcelona', 'Bari', 'Barletta', 'Basel', 'Bastia', 'Bath', 'Bayonne', 'Beaulieu-sur-Mer', 'Beaune', 'Belfast', 'Belfort', 'Belgrade', 'Bellinzona', 'Belluno', 'Benalmadena', 'Benevento', 'Benicàssim', 'Benidorm', 'Bergamo', 'Bergen', 'Bergerac', 'Berlin', 'Bern', 'Besançon', 'Béziers', 'Biarritz', 'Biel/Bienne', 'Bielefeld', 'Biella', 'Bilbao', 'Birmingham', 'Blanes', 'Bled', 'Blois', 'Bochum', 'Bodrum', 'Bologna', 'Bolzano', 'Bonifacio', 'Bonn', 'Bordeaux', 'Bordighera', 'Bormio', 'Boulogne-sur-Mer', 'Bourges', 'Bradford', 'Braga', 'Bratislava', 'Braunschweig', 'Breda', 'Bregenz', 'Brela', 'Bremen', 'Bremerhaven', 'Brescia', 'Brest', 'Briançon', 'Brighton', 'Brindisi', 'Bristol', 'Brixen', 'Brixental', 'Brno', 'Bruges', 'Brussels', 'Bucharest', 'Budapest', 'Budva', 'Burgas',  'Cadaqués', 'Cádiz', 'Caen', 'Cagliari', 'Cagnes-sur-Mer', 'Cala Bona', 'Cala d\'Or', 'Cala Millor', 'Cala Ratjada', 'Calais', 'Calella', 'Calp', 'Caltanissetta', 'Calvi', 'Cambridge', 'Cambrils', 'Campobasso', 'Can Pastilla', 'Can Picafort', 'Canazei', 'Cannes', 'Canterbury', 'Canyamel', 'Capdepera', 'Carbonia', 'Carcassonne', 'Cardiff', 'Carlisle', 'Carpi', 'Carrara', 'Cartagena', 'Caserta', 'Cassis', 'Castellón de la Plana', 'Castelrotto', 'Catania', 'Catanzaro', 'Celle', 'Cervinia', 'Cesena', 'Český Krumlov', 'Çeşme', 'Chamonix', 'Chania', 'Charleroi', 'Chartres', 'Chelmsford', 'Chemnitz', 'Cherbourg', 'Chester', 'Chiavari', 'Chichester', 'Chieti', 'Chioggia', 'Chios', 'Chișinău', 'Chur', 'Ciutadella de Menorca', 'Civitavecchia', 'Clermont-Ferrand', 'Coimbra', 'Collioure', 'Colmar', 'Cologne', 'Como', 'Conil de la Frontera', 'Copenhagen', 'Cordoba', 'Corfu', 'Corinth', 'Cork', 'Corralejo', 'Cortina d\'Ampezzo', 'Cosenza', 'Costa Adeje', 'Cottbus', 'Courchevel', 'Courmayeur', 'Coventry', 'Crans-Montana', 'Cremona', 'Crotone', 'Cuenca', 'Cuneo',  'Dachau', 'Dachstein-West', 'Darmstadt', 'Daugavpils', 'Davos', 'Deauville', 'Delft', 'Den Bosch', 'Denia', 'Derby', 'Derry', 'Didim', 'Dieppe', 'Dijon', 'Dolomiti Superski', 'Dordrecht', 'Dorfgastein', 'Dortmund', 'Dresden', 'Dublin', 'Dubrovnik', 'Duisburg', 'Dundalk', 'Dundee', 'Dunkirk', 'Durham', 'Düsseldorf',  'Eastbourne', 'Edinburgh', 'Eindhoven', 'Elche', 'Ellmau', 'Elm', 'Empuriabrava', 'Engelberg', 'Enna', 'Enschede', 'Épinal', 'Erfurt', 'Erlangen', 'Esbjerg', 'Essen', 'Estepona', 'Exeter', 'Èze',  'Faenza', 'Famagusta', 'Fano', 'Faro', 'Fermo', 'Ferrara', 'Fethiye', 'Fieberbrunn', 'Filzmoos', 'Finale Ligure', 'Fiumicino', 'Flaine', 'Florence', 'Foggia', 'Folgarida', 'Forlì', 'Forte dei Marmi', 'Frankfurt am Main', 'Freiburg', 'Fribourg', 'Frosinone', 'Fuengirola', 'Funchal', 'Galtür', 'Galway', 'Gandia', 'Gdansk', 'Gdynia', 'Gelsenkirchen', 'Geneva', 'Genoa', 'Ghent', 'Gijón', 'Girona', 'Glasgow', 'Gloucester', 'Gorizia', 'Gothenburg', 'Göttingen', 'Gouda', 'Granada', 'Granville', 'Grasse', 'Graz', 'Grenoble', 'Grindelwald', 'Groningen', 'Grossarl', 'Grosseto', 'Gstaad', 'Guimarães',  'Haarlem', 'Halle', 'Hamburg', 'Hannover', 'Hasselt', 'Hastings', 'Heidelberg', 'Heilbronn', 'Heiligenblut', 'Helsinki', 'Heraklion', 'Herceg Novi', 'Hereford', 'Hinterglemm', 'Hinterstoder', 'Hintertux', 'Hochkönig', 'Honfleur', 'Horsens', 'Hvar', 'Hyères',  'Ibiza Town', 'Imola', 'Imperia', 'Inca', 'Ingolstadt', 'Innsbruck', 'Interlaken', 'Inverness', 'Ioannina', 'Ischgl', 'Isernia', 'Istanbul', 'İzmir', 'Izola',  'Jena', 'Jerez de la Frontera', 'Juan-les-Pins', 'Jungfrau', 'Jūrmala',  'Kaiserslautern', 'Kalamata', 'Kappl', 'Kaprun', 'Karlovy Vary', 'Karlsruhe', 'Kassel', 'Kastoria', 'Katschberg', 'Kaunas', 'Kavala', 'Kemer', 'Kiel', 'Kilkenny', 'Kingston upon Hull', 'Kitzbühel', 'Klagenfurt', 'Klaipėda', 'Koblenz', 'Kolding', 'Komotini', 'Koper', 'Kos', 'Košice', 'Kotor', 'Krakow', 'Krefeld', 'Kreischberg', 'Kuşadası', 'Kutná Hora', 'Kyiv', 'Kyrenia',  'La Ciotat', 'La Clusaz', 'La Laguna', 'La Maddalena', 'La Manga', 'La Plagne', 'La Rochelle', 'La Seyne-sur-Mer', 'La Spezia', 'La Thuile', 'Laax', 'Lachtal', 'Lagos', 'Lamezia Terme', 'Lancaster', 'Landshut', 'L\'Aquila', 'Larnaca', 'Las Palmas', 'Latina', 'Lausanne', 'Le Havre', 'Le Lavandou', 'Le Mans', 'Le Puy-en-Velay', 'Lecce', 'Lecco', 'Lech', 'Leeds', 'Legnano', 'Leicester', 'Leiden', 'Leipzig', 'Lemgo', 'Leogang', 'Les Arcs', 'Les Deux Alpes', 'Les Gets', 'Les Houches', 'Les Menuires', 'Leuven', 'Leverkusen', 'Liberec', 'Lichfield', 'Liège', 'Lienz', 'Liepāja', 'Lille', 'Limassol', 'Limerick', 'Limoges', 'Lincoln', 'Lindos', 'Linz', 'Lisbon', 'Lisburn', 'Liverpool', 'Livigno', 'Livorno', 'Ljubljana', 'Lloret de Mar', 'Llucmajor', 'Loano', 'Locarno', 'Lodi', 'Lodz', 'Logroño', 'London', 'Los Cristianos', 'Lourdes', 'Loutraki', 'Louvain-la-Neuve', 'Lübeck', 'Lublin', 'Lucca', 'Lucerne', 'Lugano', 'Lund', 'Lungau', 'Luxembourg', 'Lyon',  'Maastricht', 'Macerata', 'Madonna di Campiglio', 'Madrid', 'Magaluf', 'Magdeburg', 'Mahón', 'Mainz', 'Makarska', 'Malaga', 'Malia', 'Malmö', 'Manacor', 'Manchester', 'Mannheim', 'Manosque', 'Mantua', 'Marbella', 'Maria Alm', 'Maribor', 'Marmaris', 'Marsala', 'Marseille', 'Martigues', 'Maspalomas', 'Massa', 'Matera', 'Mayrhofen', 'Mazara del Vallo', 'Mechelen', 'Megève', 'Menton', 'Merano', 'Meribel', 'Mérida', 'Messina', 'Mestre', 'Metz', 'Middelburg', 'Mijas', 'Milan', 'Millau', 'Minsk', 'Modena', 'Modica', 'Moena', 'Monaco City', 'Mönchengladbach', 'Mons', 'Monschau', 'Montafon', 'Monte Carlo', 'Monte Rosa', 'Montepulciano', 'Montluçon', 'Montpellier', 'Montreux', 'Monza', 'Moraira', 'Morzine', 'Moscow', 'Mulhouse', 'Munich', 'Münster', 'Murcia', 'Murter', 'Mykonos', 'Mytilene',  'Nafplio', 'Namur', 'Nancy', 'Nantes', 'Naples', 'Narbonne', 'Narva', 'Nassfeld', 'Naxos', 'Nerja', 'Nevers', 'Newcastle', 'Newport', 'Newry', 'Nice', 'Nicosia', 'Nijmegen', 'Nîmes', 'Niort', 'Norwich', 'Nottingham', 'Novara', 'Novigrad', 'Nuoro', 'Nürnberg', 'Nuuk', 'Nyon',  'Obergurgl', 'Oberhausen', 'Obertauern', 'Odense', 'Oia', 'Olbia', 'Oldenburg', 'Olomouc', 'Opatija', 'Oristano', 'Orléans', 'Ortisei', 'Oslo', 'Osnabrück', 'Ostrava', 'Oulu', 'Ourense', 'Oviedo', 'Oxford', 'Paderborn', 'Padova', 'Palanga', 'Palavas-les-Flots', 'Palermo', 'Palma de Mallorca', 'Palma Nova', 'Pamplona', 'Paphos', 'Paralia', 'Parikia', 'Paris', 'Parma', 'Pärnu', 'Passo del Tonale', 'Passo Rolle', 'Patras', 'Pau', 'Pavia', 'Peguera', 'Peniscola', 'Perast', 'Périgueux', 'Perpignan', 'Perros-Guirec', 'Perth (UK)', 'Perugia', 'Pesaro', 'Pescara', 'Pescasseroli', 'Peterborough', 'Petra', 'Petrovac', 'Pforzheim', 'Piacenza', 'Piraeus', 'Piran', 'Pisa', 'Pistoia', 'Pitztal', 'Playa Blanca', 'Playa de las Américas', 'Playa de Muro', 'Plovdiv', 'Plymouth', 'Plzeň', 'Podgorica', 'Poitiers', 'Pollença', 'Pontevedra', 'Pordenone', 'Poreč', 'Port d\'Andratx', 'Portimão', 'Porto', 'Porto Cervo', 'Porto Cristo', 'Porto Torres', 'Portocolom', 'Portoferraio', 'Portofino', 'Portorož', 'Porto-Vecchio', 'Portsmouth', 'Positano', 'Potenza', 'Potsdam', 'Poznan', 'Pozzuoli', 'Prague', 'Praia da Rocha', 'Prato', 'Preston', 'Pristina', 'Propriano', 'Protaras', 'Puerto de la Cruz', 'Puerto Rico de Gran Canaria', 'Pula',  'Quarteira', 'Quimper',  'Ragusa', 'Randers', 'Rapallo', 'Ratingen', 'Ravello', 'Ravenna', 'Regensburg', 'Reggio Calabria', 'Reggio Emilia', 'Reims', 'Rennes', 'Rethymno', 'Reus', 'Reutlingen', 'Reykjavik', 'Rhodes', 'Rieti', 'Riga', 'Rijeka', 'Rimini', 'Riomaggiore', 'Rocamadour', 'Rodez', 'Rome', 'Ronda', 'Roskilde', 'Rostock', 'Rotterdam', 'Roubaix', 'Rouen', 'Rovigo', 'Rovinj',  'Sa Coma', 'Sa Pobla', 'Saalbach', 'Saarbrücken', 'Saas-Fee', 'Saint Petersburg', 'Saint-Brieuc', 'Sainte-Maxime', 'Saintes-Maries-de-la-Mer', 'Saint-Étienne', 'Saint-Jean-Cap-Ferrat', 'Saint-Malo', 'Saint-Tropez', 'Salamanca', 'Salerno', 'Salisbury', 'Salou', 'Salzburg', 'San Marino', 'San Sebastián', 'Sanremo', 'Sant Antoni de Portmany', 'Santa Cruz de Tenerife', 'Santa Eulària des Riu', 'Santa Margherita Ligure', 'Santa Pola', 'Santa Ponsa', 'Santander', 'Santanyí', 'Santiago de Compostela', 'Sarajevo', 'S\'Arenal', 'Sarlat-la-Canéda', 'Sassari', 'Saumur', 'Savona', 'Schaffhausen', 'Schladming', 'Seefeld', 'Segovia', 'Semmering', 'Serre Chevalier', 'Sète', 'Seville', 'Sheffield', 'Šiauliai', 'Šibenik', 'Side', 'Siegen', 'Siena', 'Sineu', 'Sion', 'Sitges', 'Skiathos', 'SkiWelt', 'Skopje', 'Sofia', 'Sölden', 'Söll', 'Soller', 'Solothurn', 'Sondrio', 'Sopot', 'Sorrento', 'Southampton', 'Split', 'Sportgastein', 'St Albans', 'St. Anton', 'St. Gallen', 'St. Moritz', 'Stavanger', 'Stirling', 'Stockholm', 'Stoke-on-Trent', 'Strasbourg', 'Stubai', 'Stuhleck', 'Stuttgart', 'Sunderland', 'Sveti Stefan', 'Swansea', 'Syracuse', 'Szczecin',  'Tallinn', 'Tampere', 'Taormina', 'Taranto', 'Tarifa', 'Tarragona', 'Tartu', 'Tauplitz', 'Tavira', 'Teramo', 'Terni', 'The Hague', 'Thessaloniki', 'Tignes', 'Tilburg', 'Tinos', 'Tirana', 'Tivat', 'Tivoli', 'Toledo', 'Torre del Greco', 'Torre del Mar', 'Torremolinos', 'Torrevieja', 'Tórshavn', 'Toruń', 'Tossa de Mar', 'Toulon', 'Toulouse', 'Tours', 'Trani', 'Trapani', 'Trento', 'Treviso', 'Trier', 'Trieste', 'Trogir', 'Tromsø', 'Trondheim', 'Trouville-sur-Mer', 'Troyes', 'Tui', 'Turin', 'Turku', 'Turracher Höhe',  'Udine', 'Ulcinj', 'Ulm', 'Umag', 'Uppsala', 'Urbino', 'Utrecht',  'Val d’Isère', 'Val di Fassa', 'Val Gardena', 'Val Thorens', 'Valence', 'Valencia', 'Valladolid', 'Valldemossa', 'Valle Isarco', 'Valletta', 'Varazze', 'Varese', 'Varna', 'Vejle', 'Venice', 'Ventimiglia', 'Ventspils', 'Verbania', 'Verbier', 'Vercelli', 'Verona', 'Versailles', 'Vevey', 'Viareggio', 'Vibo Valentia', 'Viborg', 'Vicenza', 'Vichy', 'Vienna', 'Vigo', 'Vilamoura', 'Villach', 'Villefranche-sur-Mer', 'Vilnius', 'Viterbo', 'Vitoria-Gasteiz', 'Volos', 'Vrsar',  'Wakefield', 'Warsaw', 'Warth', 'Waterford', 'Wengen', 'Westendorf', 'Wiesbaden', 'Winchester', 'Winterthur', 'Wolfsburg', 'Wolverhampton', 'Worcester', 'Worthing', 'Wroclaw', 'Wuppertal', 'Würzburg',   'York', 'd', 'Zadar', 'Zagreb', 'Zakopane', 'Zaragoza', 'Zell am See', 'Zell am Ziller', 'Zermatt', 'Zillertal', 'Zug', 'Zurich', 'Zwickau', 'Zwolle'])

Suppose we would like to know the length of the longest name in our list.

First, we need a function that receives a string and returns its length (hint: you can use the [len](https://docs.python.org/3.8/library/functions.html#len) Python function).

In [10]:
def length(s):
    # Your code goes here
    return (len(s),s)

Now define another function that, given two numbers, returns the maximum.

In [11]:
def maximum(x, y):
    # Your code goes here
    return x if x[0] > y[0] else y

Finally, combine the two functions you just defined in a Map-Reduce chain to obtain the desired result.

In [12]:
# Your code goes here
names.map(length).reduce(maximum)

(27, 'Puerto Rico de Gran Canaria')

### Bonus: What's the name of that city?

Think how you can make the length function return both length and element name, and select the longest one in the reduction.

## 2. Map-Reduce on text files: Word Count

A typical real-life example where the Map-Reduce pattern is applied is the processing of text files like, for instance, logs generated by a web server.

In this second part of the exercise, you will process a text file and count the occurrences of each word in it.

### Prepare input data
First we will download the input file for this exercise, which corresponds to the Complete Works of William Shakespeare, offered by [Project Gutenberg](http://www.gutenberg.org/).

In [13]:
!curl  -o 100.zip https://swanserver.web.cern.ch/swanserver/csc/100.zip
!unzip -o 100.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1988k  100 1988k    0     0  5230k      0 --:--:-- --:--:-- --:--:-- 5231k
Archive:  100.zip
  inflating: 100.txt                 


Next we will use Spark to create a collection out of this text file. In the collection, every line of the file will be an element.

In [14]:
textFile = sc.textFile('100.txt')

`textFile` is a collection where each element is a string corresponding to a line in the text file.

### Introducing new operations

We will use a Map-Reduce chain to count the ocurrences of each word in the file that is a bit more complicated than what we have seen so far. The chain will contain **two** map operations and one reduce. In addition, you will have to use some new operators, which are a slight variation of the `map` and `reduce` you already know:

* [flatMap](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.flatMap.html#pyspark.RDD.flatMap): If the user function $F_{map}$ returns a collection of values instead of single value, `flatMap` decomposes the collection into individual elements in the final result.

<img align=left src="https://cdn.rawgit.com/jkthompson/pyspark-pictures/master/images/pyspark-page4.svg" width=500 height=250/>

<br clear="all">

* [reduceByKey](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.reduceByKey.html#pyspark.RDD.reduceByKey): Like in the case of `reduce`, the function $F_{reduce}$ passed as parameter of `reduceByKey` receives two elements of type $Y$ and returns an element of the same type. However, `reduceByKey` is applied on collections of key-value pairs, that is, $( ~(key_{1},val_{x}),~(key_{2},val_{y}),~(key_{1},val_{z}),~...~)$, and the output is the reduction of the values for each key $( ~(key_{1}, red_{1}),~(key_{2},red_{2}),~...~)$.

<img align=left src="https://cdn.rawgit.com/jkthompson/pyspark-pictures/master/images/pyspark-page44.svg" width=500 height=250/>

### Functions

Let's now define the functions that will be applied in the Map-Reduce chain. First, you will need a function that splits a line of the text file in a list of words (hint: you can use the [split](https://docs.python.org/2/library/stdtypes.html#str.split) Python function and return words with non alphanumeric characters).

In [15]:
def splitWords(line):
    # Your code goes here
    return line.split()

The second function you will need receives a word and returns a key-value pair, where the key is the word and the value is 1 (its partial count).

In [16]:
def count(word):
    # Your code goes here
    return (word, 1)

Finally, you will need a function that receives two numbers and returns their sum.

In [17]:
def add(x, y):
    # Your code goes here
    return x + y

### Map-Reduce chain

To finish this exercise, combine the three functions you just defined in a chain to get the counts per word in the file.

In [18]:
wordCounts = textFile.flatMap(splitWords).map(count).reduceByKey(add)

Let's see if you got it right...

In [19]:
wordCounts.collect()

[('The', 3977),
 ('Project', 85),
 ('EBook', 2),
 ('of', 15649),
 ('Shakespeare', 45),
 ('is', 7874),
 ('use', 266),
 ('anyone', 4),
 ('anywhere', 4),
 ('at', 2227),
 ('no', 2439),
 ('restrictions', 2),
 ('whatsoever.', 3),
 ('may', 1341),
 ('it,', 529),
 ('give', 964),
 ('away', 294),
 ('re-use', 2),
 ('this', 4809),
 ('online', 4),
 ('www.gutenberg.org', 2),
 ('COPYRIGHTED', 1),
 ('eBook,', 2),
 ('Details', 1),
 ('Below', 3),
 ('Please', 59),
 ('copyright', 21),
 ('guidelines', 1),
 ('in', 9633),
 ('file.', 1),
 ('Author:', 1),
 ('Posting', 1),
 ('Date:', 2),
 ('1,', 1),
 ('2011', 1),
 ('January,', 1),
 ('1994', 1),
 ('Language:', 1),
 ('English', 120),
 ('Character', 1),
 ('set', 396),
 ('***', 8),
 ('OF', 1500),
 ('THIS', 11),
 ('GUTENBERG', 224),
 ('SHAKESPEARE', 225),
 ('World', 4),
 ('Library,', 3),
 ('Inc.,', 3),
 ('Future', 3),
 ('Etext', 2),
 ('presented', 11),
 ('Gutenberg,', 1),
 ('cooperation', 1),
 ('are', 2940),
 ('placed', 10),
 ('Public', 2),
 ('Domain!!', 1),
 ('*This

### Bonus: Sort by occurrence
Now that you hopefully got the word count correct, try Python's [`sorted(iterable, key=..., reverse=True/False)`](https://docs.python.org/3/library/functions.html#sorted) to find the most frequent word in our sample. What is it?

In [20]:
sorted(wordCounts.collect(), key=lambda x: x[1], reverse=True)

[('the', 23407),
 ('I', 19540),
 ('and', 18358),
 ('to', 15682),
 ('of', 15649),
 ('a', 12586),
 ('my', 10825),
 ('in', 9633),
 ('you', 9129),
 ('is', 7874),
 ('that', 7543),
 ('And', 7068),
 ('not', 6967),
 ('with', 6771),
 ('his', 6218),
 ('be', 6017),
 ('your', 6016),
 ('for', 5629),
 ('have', 5236),
 ('it', 4920),
 ('me', 4832),
 ('this', 4809),
 ('he', 4548),
 ('as', 4280),
 ('thou', 4247),
 ('will', 4055),
 ('The', 3977),
 ('thy', 3630),
 ('but', 3565),
 ('him', 3087),
 ('To', 3068),
 ('shall', 3018),
 ('so', 2962),
 ('are', 2940),
 ('all', 2934),
 ('do', 2846),
 ('by', 2840),
 ('her', 2822),
 ('That', 2734),
 ('our', 2692),
 ('we', 2491),
 ('on', 2453),
 ('no', 2439),
 ('But', 2322),
 ('from', 2298),
 ('at', 2227),
 ('good', 2046),
 ('what', 2020),
 ('am', 1995),
 ('would', 1974),
 ('their', 1936),
 ('Enter', 1926),
 ('was', 1890),
 ('they', 1855),
 ('A', 1850),
 ('if', 1830),
 ('What', 1814),
 ('or', 1794),
 ('thee', 1780),
 ("I'll", 1737),
 ('For', 1699),
 ('If', 1642),
 ('My'

Of course it would be "the" ...